In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

In [2]:
Input=pd.read_csv("Spotify Million Song Dataset_exported.csv")
df=Input.drop(['link'],axis=1)
Input.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
max_len_word=len(df['text'][0][0])
for songs in df['text']:
    for words in songs:
        if len(words)>max_len_word:
            max_len_word=len(words)
max_len_word

#104

1

In [5]:
max_words = 674
max_len = 104

In [11]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(sequences, maxlen=max_len)

In [10]:
# Convert song names to numerical labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['song'])

In [12]:
# Model architecture
model = Sequential([
    Embedding(max_words, 32, input_length=max_len),
    Bidirectional(LSTM(32)),
    Dense(df['song'].nunique(), activation='softmax')
])

In [ ]:
#Model 2
# model2 = Sequential([
#     Embedding(max_words, df['song'].nunique(), input_length=max_len),
#     Bidirectional(LSTM(df['song'].nunique())),
#     Dense(df['song'].nunique(), activation='softmax')
# ])

In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(X, y, epochs=10, validation_split=0.2)

Epoch 1/10
1442/1442 [==============================] - 68s 45ms/step - loss: 10.8212 - accuracy: 1.9514e-04 - val_loss: 11.1312 - val_accuracy: 0.0000e+00
Epoch 2/10
1442/1442 [==============================] - 66s 46ms/step - loss: 10.5490 - accuracy: 5.4206e-04 - val_loss: 11.6262 - val_accuracy: 4.3365e-04
Epoch 3/10
1442/1442 [==============================] - 67s 46ms/step - loss: 10.3849 - accuracy: 0.0012 - val_loss: 12.4170 - val_accuracy: 0.0012
Epoch 4/10
1442/1442 [==============================] - 67s 47ms/step - loss: 9.8289 - accuracy: 0.0044 - val_loss: 13.1068 - val_accuracy: 0.0031
Epoch 5/10
1442/1442 [==============================] - 67s 47ms/step - loss: 8.9446 - accuracy: 0.0111 - val_loss: 13.6695 - val_accuracy: 0.0046
Epoch 6/10
1442/1442 [==============================] - 67s 47ms/step - loss: 8.1278 - accuracy: 0.0225 - val_loss: 14.3909 - val_accuracy: 0.0057
Epoch 7/10
1442/1442 [==============================] - 67s 46ms/step - loss: 7.4183 - accuracy: 0.

In [16]:
df['text'][0]

"Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n"

In [19]:
# Prediction
input_snippet = df['text'][0]#"Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?"
input_sequence = tokenizer.texts_to_sequences([input_snippet])
input_sequence_padded = pad_sequences(input_sequence, maxlen=max_len)
predicted_index = np.argmax(model.predict(input_sequence_padded))
predicted_song = df['song'].unique()[predicted_index]

print(f'Predicted Song: {predicted_song}')

1/1 [==============================] - 0s 19ms/step
Predicted Song: Now They Call It Swing
